In [30]:
import pandas as pd
import numpy as np
import os
import pickle

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedShuffleSplit, cross_val_predict, RandomizedSearchCV

# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report, confusion_matrix, matthews_corrcoef
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold

def split_train_test(X,y, test_ratio):
    split = StratifiedShuffleSplit(n_splits=1, test_size=test_ratio, random_state=1)
    for train_idx, test_idx in split.split(X, y):
        X_train = X.loc[train_idx]
        X_test = X.loc[test_idx]
        y_train = y.loc[train_idx]
        y_test = y.loc[test_idx]
    return X_train, X_test, y_train, y_test

In [8]:
specs = pd.read_csv('../concat_data/species_concat.csv')
specs = specs[lambda x: x['분류군명'] == '무척추동물']
specs = specs[lambda x: (x['국가적색목록.등급'] != '국가적색목록 미평가(NE)') & 
              (x['국가적색목록.등급'] != '국가적색목록 관심대상(LC)') & 
            (x['국가적색목록.등급'] != '국가적색목록 정보부족(DD)')]

In [10]:
csv_list = []
insect_dir = [
    '/Users/shinbo/Desktop/contest/environ/concat_data/bnin/etc/0.5%~2%/',
    '/Users/shinbo/Desktop/contest/environ/concat_data/bnin/etc/2%~5%/',
    '/Users/shinbo/Desktop/contest/environ/concat_data/bnin/etc/5%~/'
]

ratio = ['low','middle','high']

candi_csv = dict()
for rat in ratio:
    candi_csv[rat] = []
for rat, dir_ in zip(ratio, insect_dir):
    temp = sorted(os.listdir(dir_))[1:]
    candi_csv[rat] += [i[:-4] for i in temp]
    
name = []
for rat, dir_ in zip(ratio, insect_dir):
    temp = set(candi_csv[rat]) & set(specs['국명'])
    if len(temp) == 0:
        continue
    else:
        csv_list += [dir_ + i + '.csv' for i in temp]
        name += temp

In [13]:
csv_list

['/Users/shinbo/Desktop/contest/environ/concat_data/bnin/etc/0.5%~2%/엷은재첩.csv',
 '/Users/shinbo/Desktop/contest/environ/concat_data/bnin/etc/0.5%~2%/대칭이.csv',
 '/Users/shinbo/Desktop/contest/environ/concat_data/bnin/etc/0.5%~2%/참다슬기.csv',
 '/Users/shinbo/Desktop/contest/environ/concat_data/bnin/etc/2%~5%/좀주름다슬기.csv',
 '/Users/shinbo/Desktop/contest/environ/concat_data/bnin/etc/5%~/재첩.csv',
 '/Users/shinbo/Desktop/contest/environ/concat_data/bnin/etc/5%~/왕우렁이.csv',
 '/Users/shinbo/Desktop/contest/environ/concat_data/bnin/etc/5%~/주름다슬기.csv',
 '/Users/shinbo/Desktop/contest/environ/concat_data/bnin/etc/5%~/왼돌이물달팽이.csv',
 '/Users/shinbo/Desktop/contest/environ/concat_data/bnin/etc/5%~/강우렁이.csv']

## random forest

In [17]:
def rf_gridSearch_train(X_train, y_train, save, file_dir=None):
    param_grid = {'bootstrap': [True, False],
     'max_depth': [ 80, 90, 100, None],
     'max_features': ['auto', 'sqrt'],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10],
     'n_estimators': [600, 800, 1000]
                 }

    # Create a based model
    rf = RandomForestClassifier()
    # Instantiate the grid search model
    grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                              cv = 3, n_jobs = -1, verbose = 2)

    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)

    # best params

    rf_best_est = grid_search.best_estimator_
    rf = rf_best_est.fit(X_train, y_train)

    rf.fit(X_train, y_train)
    
    if save:
        pickle.dump(rf, open(file_dir, 'wb'))
    
    return rf

def print_result(model, X_test, y_test):
    pred = model.predict(X_test)

    print(classification_report(y_test, pred))
    
    print('ROC/AUC: ', roc_auc_score(y_test, pred))
    print('ACC: ', accuracy_score(y_test, pred))
    print('PREC: ', precision_score(y_test, pred))
    print('RECALL: ', recall_score(y_test, pred))
    print('F1: ', f1_score(y_test, pred))
    print('MCC: ', matthews_corrcoef(y_test, pred))

    print('Confusion Matrix: \n', confusion_matrix(y_test, pred))

In [ ]:

for n, _dir in zip(name, csv_list):
    
    tmp = pd.read_csv(_dir)
    X = tmp[['bio1', 'bio2', 'bio3', 'bio4', 'bio5', 'bio6', 'bio7',
           'bio8', 'bio9', 'bio10', 'bio11', 'bio12', 'bio13', 'bio14', 'bio15',
           'bio16', 'bio17', 'bio18', 'bio19', 'pet', 'rh', 'srad', 'lon', 'lat',
           'alt']]
    y = tmp['y']

    X_train, X_test, y_train, y_test = split_train_test(X, y,0.3)

    rf = rf_gridSearch_train(X_train, y_train, True, file_dir= _dir[:-4] + '_rf.sav')
    dict_model_rf[ n ] = rf
    
    print('\n')
    print(f'########################### classification report for {n} ###########################')
    print_result(rf,X_test,y_test)
    print('\n')

Fitting 3 folds for each of 432 candidates, totalling 1296 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 150 tasks      | elapsed:   39.5s


## over 20%

In [14]:
_dir = '/Users/shinbo/Desktop/contest/environ/concat_data/flr/etc/5%~/'
list_dir = sorted(os.listdir(_dir))

list_csv = []
for csv in list_dir:
    if csv[-3:] !='csv':
        continue
    temp = pd.read_csv(_dir + csv)
    if np.mean(temp['y']) > 0.2:
        list_csv.append(csv)

In [15]:
list_csv

['개망초.csv',
 '국수나무.csv',
 '닭의장풀.csv',
 '담쟁이덩굴.csv',
 '댕댕이덩굴.csv',
 '붉나무.csv',
 '사위질빵.csv',
 '산딸기.csv',
 '산초나무.csv',
 '생강나무.csv',
 '소나무.csv',
 '쑥.csv',
 '으름덩굴.csv',
 '작살나무.csv',
 '주름조개풀.csv',
 '쥐똥나무.csv',
 '찔레나무.csv',
 '청미래덩굴.csv',
 '칡.csv']

### logistic

In [20]:
name = [i[:-4] for i in list_csv]
dict_model_rf = {}
save_dir = '/Users/shinbo/Desktop/contest/environ/model/flr/etc/'
dict_model = {}
for n, d in zip(name, list_csv):
    
    tmp = pd.read_csv(_dir + d)

    X = tmp[['bio1', 'bio2', 'bio3', 'bio4', 'bio5', 'bio6', 'bio7',
           'bio8', 'bio9', 'bio10', 'bio11', 'bio12', 'bio13', 'bio14', 'bio15',
           'bio16', 'bio17', 'bio18', 'bio19', 'pet', 'rh', 'srad', 'lon', 'lat',
           'alt']]
    y = tmp['y']

    X_train, X_test, y_train, y_test = split_train_test(X, y,0.3)


    # define model
    #weights = {0:0.01, 1:1.0}
    model = LogisticRegression(solver='lbfgs', class_weight='balanced')
    model.fit(X_train, y_train)
    print(f'########################### classification report for {n} ###########################')
    print_result(model, X_test, y_test)
    print('\n')
    
    dict_model[n] = model

/Users/shinbo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


########################### classification report for 개망초 ###########################
              precision    recall  f1-score   support

           0       0.83      0.45      0.58       214
           1       0.25      0.67      0.37        60

    accuracy                           0.50       274
   macro avg       0.54      0.56      0.47       274
weighted avg       0.70      0.50      0.53       274

ROC/AUC:  0.557632398753894
ACC:  0.49635036496350365
PREC:  0.25316455696202533
RECALL:  0.6666666666666666
F1:  0.36697247706422015
MCC:  0.09647652794725824
Confusion Matrix: 
 [[ 96 118]
 [ 20  40]]




/Users/shinbo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


########################### classification report for 국수나무 ###########################
              precision    recall  f1-score   support

           0       0.80      0.52      0.63       211
           1       0.26      0.57      0.36        63

    accuracy                           0.53       274
   macro avg       0.53      0.54      0.49       274
weighted avg       0.68      0.53      0.57       274

ROC/AUC:  0.5440081245768449
ACC:  0.5291970802919708
PREC:  0.2608695652173913
RECALL:  0.5714285714285714
F1:  0.3582089552238805
MCC:  0.07407392576724163
Confusion Matrix: 
 [[109 102]
 [ 27  36]]




/Users/shinbo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


...

/Users/shinbo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


########################### classification report for 찔레나무 ###########################
              precision    recall  f1-score   support

           0       0.79      0.69      0.73       204
           1       0.33      0.46      0.39        70

    accuracy                           0.63       274
   macro avg       0.56      0.57      0.56       274
weighted avg       0.67      0.63      0.64       274

ROC/AUC:  0.5717086834733893
ACC:  0.6277372262773723
PREC:  0.3333333333333333
RECALL:  0.45714285714285713
F1:  0.38554216867469876
MCC:  0.13110529465187992
Confusion Matrix: 
 [[140  64]
 [ 38  32]]




/Users/shinbo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


########################### classification report for 청미래덩굴 ###########################
              precision    recall  f1-score   support

           0       0.80      0.68      0.73       213
           1       0.26      0.39      0.31        61

    accuracy                           0.62       274
   macro avg       0.53      0.54      0.52       274
weighted avg       0.68      0.62      0.64       274

ROC/AUC:  0.53709689832987
ACC:  0.6167883211678832
PREC:  0.2608695652173913
RECALL:  0.39344262295081966
F1:  0.31372549019607837
MCC:  0.06535708997853872
Confusion Matrix: 
 [[145  68]
 [ 37  24]]


########################### classification report for 칡 ###########################
              precision    recall  f1-score   support

           0       0.81      0.49      0.61       212
           1       0.26      0.60      0.36        62

    accuracy                           0.51       274
   macro avg       0.53      0.54      0.48       274
weighted avg       0.68   

/Users/shinbo/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### random forest over 5% ~

In [25]:
name = [i[:-4] for i in list_csv]
dict_model_rf = {}
save_dir = '/Users/shinbo/Desktop/contest/environ/model/flr/'
for n, d in zip(name, list_csv):
    
    tmp = pd.read_csv(_dir + d)
    X = tmp[['bio1', 'bio2', 'bio3', 'bio4', 'bio5', 'bio6', 'bio7',
           'bio8', 'bio9', 'bio10', 'bio11', 'bio12', 'bio13', 'bio14', 'bio15',
           'bio16', 'bio17', 'bio18', 'bio19', 'pet', 'rh', 'srad', 'lon', 'lat',
           'alt']]
    y = tmp['y']
    print(y.value_counts())
    X_train, X_test, y_train, y_test = split_train_test(X, y,0.3)

    rf = rf_gridSearch_train(X_train, y_train, True, file_dir= save_dir + n + '_rf.sav')
    dict_model_rf[ n ] = rf
    
    print('\n')
    print(f'########################### classification report for {n} ###########################')
    print_result(rf,X_test,y_test)
    print('\n')

0    714
1    199
Name: y, dtype: int64
Fitting 3 folds for each of 432 candidates, totalling 1296 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done 150 tasks      | elapsed:   45.7s
[Parallel(n_jobs=-1)]: Done 353 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 636 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 1001 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 1296 out of 1296 | elapsed:  6.5min finished




########################### classification report for 개망초 ###########################
              precision    recall  f1-score   support

           0       0.79      0.99      0.88       214
           1       0.50      0.03      0.06        60

    accuracy                           0.78       274
   macro avg       0.64      0.51      0.47       274
weighted avg       0.72      0.78      0.70       274

ROC/AUC:  0.5119937694704051
ACC:  0.781021897810219
PREC:  0.5
RECALL:  0.03333333333333333
F1:  0.0625
MCC:  0.08270967739612899
Confusion Matrix: 
 [[212   2]
 [ 58   2]]


0    702
1    211
Name: y, dtype: int64
Fitting 3 folds for each of 432 candidates, totalling 1296 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 150 tasks      | elapsed:   43.6s
[Parallel(n_jobs=-1)]: Done 353 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 636 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 1001 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 1296 out of 1296 | elapsed:  6.4min finished




########################### classification report for 국수나무 ###########################
              precision    recall  f1-score   support

           0       0.77      0.98      0.86       211
           1       0.29      0.03      0.06        63

    accuracy                           0.76       274
   macro avg       0.53      0.50      0.46       274
weighted avg       0.66      0.76      0.68       274

ROC/AUC:  0.5040246746407884
ACC:  0.7591240875912408
PREC:  0.2857142857142857
RECALL:  0.031746031746031744
F1:  0.057142857142857134
MCC:  0.021466815545820597
Confusion Matrix: 
 [[206   5]
 [ 61   2]]


0    724
1    189
Name: y, dtype: int64
Fitting 3 folds for each of 432 candidates, totalling 1296 fits


...



########################### classification report for 찔레나무 ###########################
              precision    recall  f1-score   support

           0       0.77      0.97      0.86       204
           1       0.63      0.17      0.27        70

    accuracy                           0.76       274
   macro avg       0.70      0.57      0.56       274
weighted avg       0.74      0.76      0.71       274

ROC/AUC:  0.5685574229691877
ACC:  0.7627737226277372
PREC:  0.631578947368421
RECALL:  0.17142857142857143
F1:  0.2696629213483146
MCC:  0.23539741700226727
Confusion Matrix: 
 [[197   7]
 [ 58  12]]


0    711
1    202
Name: y, dtype: int64
Fitting 3 folds for each of 432 candidates, totalling 1296 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done 150 tasks      | elapsed:   53.1s
[Parallel(n_jobs=-1)]: Done 353 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 636 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 1001 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 1296 out of 1296 | elapsed:  8.0min finished




########################### classification report for 청미래덩굴 ###########################
              precision    recall  f1-score   support

           0       0.80      0.98      0.88       213
           1       0.64      0.15      0.24        61

    accuracy                           0.79       274
   macro avg       0.72      0.56      0.56       274
weighted avg       0.77      0.79      0.74       274

ROC/AUC:  0.5620334026014007
ACC:  0.791970802919708
PREC:  0.6428571428571429
RECALL:  0.14754098360655737
F1:  0.24
MCC:  0.23440108974901627
Confusion Matrix: 
 [[208   5]
 [ 52   9]]


0    705
1    208
Name: y, dtype: int64
Fitting 3 folds for each of 432 candidates, totalling 1296 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done 150 tasks      | elapsed:   52.6s
[Parallel(n_jobs=-1)]: Done 353 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 636 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 1001 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 1296 out of 1296 | elapsed:  8.0min finished




########################### classification report for 칡 ###########################
              precision    recall  f1-score   support

           0       0.78      0.96      0.86       212
           1       0.36      0.08      0.13        62

    accuracy                           0.76       274
   macro avg       0.57      0.52      0.50       274
weighted avg       0.68      0.76      0.70       274

ROC/AUC:  0.5190961655508216
ACC:  0.7591240875912408
PREC:  0.35714285714285715
RECALL:  0.08064516129032258
F1:  0.13157894736842105
MCC:  0.07257536857954912
Confusion Matrix: 
 [[203   9]
 [ 57   5]]




### xgb over 2% ~ 5%

In [ ]:
dir_name = '/Users/shinbo/Desktop/contest/environ/concat_data/flr/etc/2%~5%/'
dir_list = os.listdir(dir_name)
dict_model_xgb = {}
flr_data_dict = {}
save_dir = '/Users/shinbo/Desktop/contest/environ/model/flr/'
for _dir in dir_list:
    
    tmp = pd.read_csv(dir_name + _dir)
    X = tmp[['bio1', 'bio2', 'bio3', 'bio4', 'bio5', 'bio6', 'bio7',
           'bio8', 'bio9', 'bio10', 'bio11', 'bio12', 'bio13', 'bio14', 'bio15',
           'bio16', 'bio17', 'bio18', 'bio19', 'pet', 'rh', 'srad', 'lon', 'lat',
           'alt']]
    y = tmp['y']
    
    weight = len(np.where(y == 0)[0]) / len(np.where(y == 1)[0])

    X_train, X_test, y_train, y_test = split_train_test(X, y,0.3)

        # Create XGB Classifier object
    xgb_clf = XGBClassifier(scale_pos_weight = weight)

    # Create parameter grid
    parameters = {"learning_rate": [0.1, 0.01, 0.001],
                   "gamma" : [0.01, 0.1, 0.3, 0.5, 1, 1.5, 2],
                   "max_depth": [2, 4, 7, 10],
                   "colsample_bytree": [0.3, 0.6, 0.8, 1.0],
                   "subsample": [0.2, 0.4, 0.5, 0.6, 0.7],
                   "reg_alpha": [0, 0.5, 1],
                   "reg_lambda": [1, 1.5, 2, 3, 4.5],
                   "min_child_weight": [1, 3, 5, 7],
                   "n_estimators": [100, 250, 500, 1000]}

    print(f'starting optimizing for {_dir[:-4]}')
    # Create RandomizedSearchCV Object
    xgb_rscv = RandomizedSearchCV(xgb_clf, param_distributions = parameters, scoring = "roc_auc",
                                 cv = 5, random_state = 40 , n_jobs = -1, verbose = 2)
    print('done tuning')

    # Fit the model
    model_xgboost = xgb_rscv.fit(X_train, y_train)
    
    xgb_best = model_xgboost.best_estimator_
    
    final_xgb = xgb_best.fit(X_train, y_train)
    
    pred = final_xgb.predict(X_test)
    roc_score = roc_auc_score(y_test,pred)
    if roc_score >= 0.7:        
        print('\n')
        print(f'########################### classification report for {_dir[:-4]} ###########################')
        print_result(final_xgb,X_test,y_test)
        dict_model_xgb[_dir[:-4]] = final_xgb
        flr_data_dict[_dir[:-4]] = {'X_train':X_train, 'X_test':X_test, 'y_train':y_train, 'y_test':y_test}
        print('\n')

starting optimizing for 꿩의다리
done tuning
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    7.9s finished


starting optimizing for 엷은잎고광나무
done tuning
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    2.7s finished


starting optimizing for 탱자나무
done tuning
Fitting 5 folds for each of 10 candidates, totalling 50 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    2.0s finished


starting optimizing for 솜대
done tuning
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    2.7s finished


starting optimizing for 나도겨풀
done tuning
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.


In [51]:
dict_model_xgb.keys()

dict_keys(['올벚나무', '점백이천남성', '가는쇠고사리', '까마귀쪽나무', '나도히초미', '병풀', '돌토끼고사리', '속단', '개다래', '별고사리', '가래', '미나리아재비', '이스라지나무', '보춘화', '서양금혼초', '장딸기', '논뚝외풀', '멀꿀', '풀솜나물', '큰족제비고사리', '광릉용수염풀', '머귀나무', '개구리발톱', '콩짜개덩굴', '큰천남성', '유채', '좀딱취', '큰꼭두선이', '쥐보리', '개면마', '좀비비추', '덩굴곽향', '긴사상자', '남오미자', '비자나무', '까마귀베개', '정금나무', '자금우', '백량금', '큰사상자', '단풍돼지풀', '개족도리풀', '천선과나무', '소엽맥문동', '후박나무', '생달나무', '도깨비고비', '층꽃나무', '송이고랭이', '보리밥나무', '새비나무', '동백나무', '바위족제비고사리', '상산', '산자고', '자주괴불주머니', '새끼노루귀', '까마귀머루', '곰솔', '계요등', '윤노리나무', '나도밤나무', '낚시제비꽃', '숙대나무'])

### xgb over 5%~

In [61]:
dir_name = '/Users/shinbo/Desktop/contest/environ/concat_data/flr/etc/5%~/'
dir_list = os.listdir(dir_name)
save_dir = '/Users/shinbo/Desktop/contest/environ/model/flr/'
for _dir in dir_list:
    
    tmp = pd.read_csv(dir_name + _dir)
    X = tmp[['bio1', 'bio2', 'bio3', 'bio4', 'bio5', 'bio6', 'bio7',
           'bio8', 'bio9', 'bio10', 'bio11', 'bio12', 'bio13', 'bio14', 'bio15',
           'bio16', 'bio17', 'bio18', 'bio19', 'pet', 'rh', 'srad', 'lon', 'lat',
           'alt']]
    y = tmp['y']
    
    weight = len(np.where(y == 0)[0]) / len(np.where(y == 1)[0])

    X_train, X_test, y_train, y_test = split_train_test(X, y,0.3)

        # Create XGB Classifier object
    xgb_clf = XGBClassifier(scale_pos_weight = weight)

    # Create parameter grid
    parameters = {"learning_rate": [0.1, 0.01, 0.001],
                   "gamma" : [0.01, 0.1, 0.3, 0.5, 1, 1.5, 2],
                   "max_depth": [2, 4, 7, 10],
                   "colsample_bytree": [0.3, 0.6, 0.8, 1.0],
                   "subsample": [0.2, 0.4, 0.5, 0.6, 0.7],
                   "reg_alpha": [0, 0.5, 1],
                   "reg_lambda": [1, 1.5, 2, 3, 4.5],
                   "min_child_weight": [1, 3, 5, 7],
                   "n_estimators": [100, 250, 500, 1000]}

    print(f'starting optimizing for {_dir[:-4]}')
    # Create RandomizedSearchCV Object
    xgb_rscv = RandomizedSearchCV(xgb_clf, param_distributions = parameters, scoring = "roc_auc",
                                 cv = 5, random_state = 40 , n_jobs = -1, verbose = 2)
    print('done tuning')

    # Fit the model
    model_xgboost = xgb_rscv.fit(X_train, y_train)
    
    xgb_best = model_xgboost.best_estimator_
    
    final_xgb = xgb_best.fit(X_train, y_train)
    
    pred = final_xgb.predict(X_test)
    roc_score = roc_auc_score(y_test,pred)
    if roc_score >= 0.7:        
        print('\n')
        print(f'########################### classification report for {_dir[:-4]} ###########################')
        print_result(final_xgb,X_test,y_test)
        dict_model_xgb[_dir[:-4]] = final_xgb
        flr_data_dict[_dir[:-4]] = {'X_train':X_train, 'X_test':X_test, 'y_train':y_train, 'y_test':y_test}
        print('\n')

starting optimizing for 꿩의밥
done tuning
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   12.8s finished


starting optimizing for 까마귀밥나무
done tuning
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    2.6s finished


starting optimizing for 골풀
done tuning
Fitting 5 folds for each of 10 candidates, totalling 50 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  39 out of  50 | elapsed:    2.3s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    2.8s finished


starting optimizing for 애기나리
done tuning
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    3.0s finished


starting optimizing for 벚나무
done tuning
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    2.6s finished


In [62]:
pickle.dump(dict_model_xgb, open('../model_dict_weight_adj/flr_xgb.pickle', 'wb'))
pickle.dump(flr_data_dict, open('../model_dict_weight_adj/flr_data.pickle', 'wb'))

In [63]:
X.shape

(913, 25)